In [1]:
import torchvision
import torch
from PIL import Image
import numpy as np
import torchvision.transforms as T
import cv2

In [2]:
model = torch.load('fruit_det_mod.pth')
model = model.eval()
device = 'cuda:0'

In [3]:
def get_prediction(img, threshold):
    lab = {1:'apple', 2:'banana', 3:'orange'}
#     img = Image.open(img_path) # Load the image
    transform = T.Compose([T.ToTensor()]) # Defing PyTorch Transform
    img = transform(img) # Apply the transform to the image
    pred = model([img.to(device)]) # Pass the image to the model
    pred_class = [lab[i] for i in list(pred[0]['labels'].cpu().numpy())] # Get the Prediction Score
    pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].cpu().detach().numpy())] # Bounding boxes
    pred_score = list(pred[0]['scores'].cpu().detach().numpy())
    pred_t = [pred_score.index(x) for x in pred_score if x > threshold][-1] # Get list of index with score greater than threshold.
    pred_boxes = pred_boxes[:pred_t+1]
    pred_class = pred_class[:pred_t+1]
    return pred_boxes, pred_class

In [4]:
def detect_img(im, threshold=0.5, rect_th=3, text_size=1, text_th=2):
    boxes, pred_cls = get_prediction(im, threshold) # Get predictions
    img = cv2.cvtColor(im, cv2.COLOR_BGR2RGB) # Convert to RGB
    for i in range(len(boxes)):
        cv2.rectangle(img, boxes[i][0], boxes[i][1],color=(0, 255, 0), thickness=rect_th) # Draw Rectangle with the coordinates
        cv2.putText(img,pred_cls[i], boxes[i][0],  cv2.FONT_HERSHEY_SIMPLEX, text_size, (0,255,0),thickness=text_th) # Write the prediction class
    return img

In [9]:
cap = cv2.VideoCapture(r'C:\Users\janor\Downloads\videoplayback.mp4')
# cap = cv2.VideoCapture(0) #webcam
i=0
if cap.isOpened()== False:  
    print("Error opening video  file")
k = 0
while cap.isOpened(): 
    ret, frame = cap.read()
    k += 1
    if ret == True:
        try:
            if k % 100 == 0:
                im = detect_img(frame, 0.8)
                cv2.imshow('Frame', im)
            else:
                cv2.imshow('Frame', frame)
        except IndexError:
            cv2.imshow('Frame', frame)
        if cv2.waitKey(25) & 0xFF == ord('q'): 
            break
    else:  
        break
    
cap.release()  
cv2.destroyAllWindows()